In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import chromadb
import pandas as pd
from chromadb.config import Settings
import matplotlib.pyplot as plt
import seaborn as sns
import math
from tqdm.notebook import tqdm

### Load Synthetic Dataset

In [ ]:
csv_file_path = 'data/synthetic_clusters_colored.csv'
data = pd.read_csv(csv_file_path)

In [ ]:
query_color = "purple"
query_point = data[data['cluster'] == query_color][['x', 'y']].iloc[0]

In [ ]:
plt.figure(figsize=(8, 6))
for color in data['cluster'].unique().tolist():
    # Select data points belonging to each cluster/color
    cluster_data = data[data['cluster'] == color]
    plt.scatter(cluster_data['x'], cluster_data['y'], color=color, label=f'{color.capitalize()} Cluster')

# Highlight the query point
plt.scatter(query_point['x'], query_point['y'], color='darkorange', edgecolor='darkorange', label='Query', zorder=5)

plt.title('Visualization of Synthetic Clusters')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

### Setup Database

In [ ]:
client = chromadb.Client()

collections = client.list_collections()
if collections:
    client.delete_collection(collections[0].name)

In [ ]:
n_categories = [4]

index_configuraiton = {
    "lmi:epochs": "[200]",
    "lmi:model_types": "['MLP']",
    "lmi:lrs": "[0.01]",
    "lmi:n_categories": f"{str(n_categories)}",
    "lmi:kmeans": "{'verbose': False, 'seed': 2023, 'nredo': 10}",
}

collection_name = "synthetic_collection"
collection = client.create_collection(
    name=collection_name,
    metadata=index_configuraiton
)

In [ ]:
# Use batch upload just to test it out
batch_size = 25
for i in tqdm(range(0, len(data), batch_size), desc="Adding documents"):
    collection.add(
        embeddings=data[['x', 'y']].iloc[i: i + batch_size].values.tolist(),
        metadatas=[{"cluster": cluster} for cluster in data['cluster'].iloc[i: i + batch_size]],
        ids=data['id'].iloc[i: i + batch_size].values.tolist(),
    )

bucket_assignment = collection.build_index()

# Map the ids in data to buckets using bucket_labels_new_format (assuming this exists outside this function)
data['bucket'] = data['id'].map(lambda x: list(bucket_assignment.get(x, [])))
data['bucket_str'] = data['bucket'].apply(lambda x: str(x))

### Visualize buckets

In [ ]:
def plot_bucket_items(data, use_colors):
    """
    Plot the number of items in each bucket, with optional color coding by cluster.

    Parameters:
    data (DataFrame): The data frame containing the 'id', 'bucket', and 'cluster' columns.
    use_colors (bool): If True, use different colors for each cluster in the visualization.
    """
    if use_colors:
        # Count the number of points from each cluster in each mapped bucket
        bucket_cluster_counts = data.groupby(['bucket_str', 'cluster']).size().unstack(fill_value=0)
        plot_data = bucket_cluster_counts.reset_index().melt(id_vars='bucket_str', var_name='cluster', value_name='count')
        palette = {cluster: cluster for cluster in data['cluster'].unique()}
        
        # Create the bar plot with colors
        plt.figure(figsize=(10, 6))
        ax = sns.barplot(data=plot_data, x='bucket_str', y='count', hue='cluster', palette=palette)
    else:
        # Count the total number of items in each bucket
        bucket_counts = data.groupby('bucket_str').size()
        plot_data = bucket_counts.reset_index(name='count')

        # Create the bar plot without colors
        plt.figure(figsize=(10, 6))
        ax = sns.barplot(data=plot_data, x='bucket_str', y='count')
    
    plt.title('Number of Items in Each Bucket')
    plt.xlabel('Bucket')
    plt.ylabel('Count')

    # Annotate each bar with the count of elements
    for p in ax.patches:
        bar_height = int(p.get_height())
        if bar_height > 0:  # Only annotate bars with a height greater than zero
            ax.annotate(f'{bar_height}', (p.get_x() + p.get_width() / 2., bar_height),
                        ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                        textcoords='offset points')

    plt.show()

In [ ]:
plot_bucket_items(data, False)

In [ ]:
plot_bucket_items(data, True)

### Perform Qeries and visualize their bucket order 

In [ ]:
def visualize_bucket_order(data, bucket_order):
    order_labels = [str(list(bucket)) for bucket in bucket_order]  # Convert to string with brackets

    bucket_cluster_counts = data.groupby(['bucket_str', 'cluster']).size().unstack(fill_value=0)
    # Extract the counts for the specified color
    cluster_specific_counts = bucket_cluster_counts[filter_color]
    
    # Reindex the bucket counts according to the specified order
    ordered_counts = cluster_specific_counts.reindex(order_labels, fill_value=0).reset_index()
    ordered_counts.columns = ['bucket', 'count']
    
    # Create the bar plot
    plt.figure(figsize=(10, 6))
    sns.barplot(data=ordered_counts, x='bucket', y='count', color=filter_color)
    plt.title(f'Number of Items from {filter_color} Cluster in Ordered Buckets')
    plt.xlabel('Bucket')
    plt.ylabel('Count')
    plt.show()

In [ ]:
# Output from ChromaDB
expected_output = [['id70', 'id74', 'id10', 'id86', 'id29', 'id76', 'id92', 'id3', 'id23', 'id47']]

In [ ]:
%%time
filter_color = "red"
results = collection.query(
    query_embeddings=list(query_point),
    include=["metadatas", 'embeddings', 'distances'],
    where={"cluster": filter_color},
    n_results=10,
    n_buckets=1,
    bruteforce_threshold=0.0,
    constraint_weight=0.0,
    search_until_bucket_not_empty=True,
)

In [ ]:
print(results['ids'])
print(results['distances'])
print(results['metadatas'])
print(results['bucket_order'])
print(results['bruteforce_used'])
print(results['constraint_weight'])
assert results['ids'] == expected_output

In [ ]:
visualize_bucket_order(data, results['bucket_order'][0])

In [ ]:
expected_output = [['id70', 'id74', 'id10', 'id86', 'id29', 'id76', 'id92', 'id3', 'id23', 'id47']]

In [ ]:
%%time
results = collection.query(
    query_embeddings=list(query_point),
    include=["metadatas", 'embeddings', 'distances'],
    where={"cluster": filter_color},
    n_results=10,
    n_buckets=1,
    bruteforce_threshold=0.0, 
    constraint_weight=1,
    search_until_bucket_not_empty=True
)

In [ ]:
print(results['ids'])
print(results['distances'])
print(results['metadatas'])
print(results['bucket_order'])
print(results['bruteforce_used'])
print(results['constraint_weight'])
assert results['ids'] == expected_output

In [ ]:
visualize_bucket_order(data, results['bucket_order'][0])